In [28]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']
print len(estims_data)
#with open('./max/меланома_все.json') as data_file: #2
#with open('./max/рак_груди_все.json') as data_file: #0
#with open('./max/рак_простаты_все.json') as data_file: #4
with open('./max/рак_шейки_матки_все.json') as data_file: #1
#with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
print len(cancer_data)

10
230


In [5]:
FEATURES = ["views", "large", "top", "price_level", "speed_level", "count_orders", "count_orders_success", "count_reviews", "count_reviews_for_illness", "top_proc_cli", "gen_proc_cli", "top_oper_cli", "gen_oper_cli", "doc_count", "doctor_prof_count", "doctor_doc_count", "max_exp_doc", "mean_exp_doc", "max_h_index", "mean_h_index", "doc_top_diag", "doc_gen_diag", "doc_top_oper", "doc_gen_oper", "ill_cli", "lang_cli", "specialist", "experience", "variety", "price"]

In [257]:
clinic_ids = [i['id'] for i in estims_data]
#estims_data.pop(clinic_ids.index('0'))
print sorted(clinic_ids)
print sorted([i['id'] for i in cancer_data])

[u'1', u'109', u'262', u'264', u'273', u'78', u'8', u'87', u'96']
[u'1', u'100', u'101', u'102', u'103', u'104', u'105', u'106', u'107', u'108', u'109', u'110', u'111', u'112', u'113', u'114', u'119', u'122', u'123', u'124', u'125', u'126', u'127', u'128', u'129', u'130', u'131', u'132', u'133', u'134', u'135', u'136', u'137', u'138', u'139', u'140', u'141', u'142', u'143', u'144', u'145', u'146', u'147', u'148', u'149', u'150', u'151', u'152', u'153', u'156', u'158', u'159', u'16', u'160', u'161', u'162', u'163', u'164', u'165', u'166', u'167', u'169', u'17', u'170', u'171', u'172', u'173', u'174', u'175', u'176', u'177', u'178', u'179', u'180', u'181', u'182', u'183', u'184', u'185', u'186', u'187', u'188', u'190', u'192', u'20', u'205', u'208', u'209', u'21', u'210', u'211', u'212', u'213', u'214', u'215', u'216', u'217', u'218', u'22', u'220', u'221', u'222', u'223', u'224', u'225', u'226', u'227', u'228', u'229', u'23', u'230', u'231', u'232', u'233', u'234', u'235', u'236', u'238

In [6]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    X+=[int(clinic['views'])]
    X+=[int(clinic['large'])]
    X+=[int(clinic['top'])]
    X+=[int(clinic['price_level'])]
    X+=[int(clinic['speed_level'])]
    X+=[int(clinic['count_orders'])]
    X+=[int(clinic['count_orders_success'])]
    X+=[int(clinic['count_reviews'])]
    X+=[int(clinic['count_reviews_for_illness'])]
    X+=[sum([int(i['top']) for i in all_proc])] #top_proc_cli
    X+=[len(all_proc)] #gen_proc_cli
    X+=[sum([int(i['top']) for i in clinic['procedures']['operations']])] #top_oper_cli
    X+=[len(clinic['procedures']['operations'])] #gen_oper_cli
    X+=[len(clinic['doctors'])] #doc_count
    X+=[[i['degree'] for i in all_doc].count(u'Профессор')]    #doctor_prof_count
    X+=[[i['degree'] for i in all_doc].count(u'Доцент')]    #doctor_doc_count
    try:
        X+=[max([int(i['experience']) for i in all_doc])] #max_exp_doc
        X+=[np.mean([int(i['experience']) for i in all_doc])] #mean_exp_doc
    except ValueError:
        X+=[0,0]
    try:
        X+=[max([int(i['h_index']) for i in all_doc])] #max_h_index
        X+=[np.mean([int(i['h_index']) for i in all_doc])] #mean_h_index
    except ValueError:
        X+=[0,0]
    #doc_top_diag
    tops = [i['procedures']['diagnostics'] for i in all_doc]
    ids_diad = [i['id'] for i in sum(tops,[]) if i['top']=='1']
    X+=[len(list(set(ids_diad)))]
    #doc_gen_diag
    tops = [i['procedures']['diagnostics'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[])]
    X+=[len(list(set(ids)))]
    #doc_top_oper
    tops = [i['procedures']['operations'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[]) if i['top']=='1']
    X+=[len(list(set(ids)))]
    #doc_gen_oper
    tops = [i['procedures']['operations'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[])]
    X+=[len(list(set(ids)))]
    #ill_cli
    tops = [i['illnesses'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[])]
    X+=[len(list(set(ids)))]
    #lang_cli
    tops = [i['languages'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[])]
    X+=[len(list(set(ids)))]
    X+=[int(clinic_estim['specialist'])]
    X+=[int(clinic_estim['experience'])]
    X+=[int(clinic_estim['variety'])]
    X+=[int(clinic_estim['price'])]
    y=int(clinic_estim['rating'])
    return [X, y]

In [7]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = np.array([i[0] for i in t_cleaned_data])
    y = np.array([i[1] for i in t_cleaned_data])
    return X,y

In [8]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=False, normalize=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [29]:
X, y = get_X_y_from(cancer_data)

In [2]:
for i in range(1,10):
    regr = process_with(X, y, info=True, ts=i/10.0)
    print

NameError: name 'X_1' is not defined

In [22]:
regr = process_with(X, y, info=True, ts=0.3)

Total: 10, train: 7, test: 3
Residual sum of squares: 288.86
Train absolute: 0.00
Test absolute: 14.02
Absolute to mean: 210.37%
Train variance score: 1.00
Test variance score: -98.99


In [23]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y)):
    print "Real: %f \t Predicted: %f" %(y[i], regr.predict(X[i]))

Real: 10.000000 	 Predicted: 10.000000
Real: 9.000000 	 Predicted: 25.197154
Real: 8.000000 	 Predicted: 8.000000
Real: 7.000000 	 Predicted: 7.000000
Real: 6.000000 	 Predicted: 4.669362
Real: 5.000000 	 Predicted: 29.545328
Real: 4.000000 	 Predicted: 4.000000
Real: 3.000000 	 Predicted: 3.000000
Real: 2.000000 	 Predicted: 2.000000
Real: 1.000000 	 Predicted: 1.000000


In [ ]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    #print "%4f \t %s" %(i[0], i[1])
    print i[0]

In [ ]:
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [20]:
plt.plot([i[13] for i in X])
plt.show()

In [30]:
print FEATURES[13]
[i[13] for i in X]

doc_count


[4.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [19]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)

views 10
large 8
top 2
price_level 9
speed_level 9
count_orders 10
count_orders_success 9
count_reviews 9
count_reviews_for_illness 2
top_proc_cli 10
gen_proc_cli 10
top_oper_cli 6
gen_oper_cli 10
doc_count 5
doctor_prof_count 3
doctor_doc_count 1
max_exp_doc 4
mean_exp_doc 4
max_h_index 5
mean_h_index 5
doc_top_diag 4
doc_gen_diag 5
doc_top_oper 0
doc_gen_oper 5
ill_cli 5
lang_cli 1
